Euclid GC Fisher Matrix Code

Author: Alkistis Pourtsidou, ICG Portsmouth

Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html 

To run this Jupyter notebook you need to have CAMB and the CAMB python package 
installed. In order to install the CAMB python package on your computer follow
the instructions in http://camb.readthedocs.io/en/latest/

In [1]:
%matplotlib inline
import sys, platform, os

from matplotlib import pyplot as plt
import numpy as np

print('Using CAMB installed at '+ os.path.realpath(os.path.join(os.getcwd(),'..')))
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
import camb
from camb import model, initialpower

Using CAMB installed at /Users/alkistis/Dropbox


In [2]:
import scipy
from scipy.interpolate import interp1d
from __future__ import division

In [3]:
from scipy import integrate
from scipy import linalg

pi=np.pi

In [4]:
#Fiducial cosmological parameters

hubble=0.67
omegab=0.022445/hubble**2
omegac=0.121203/hubble**2
om0=omegac+omegab
H00=100*hubble
Ass=2.1265e-9
nss=0.96

kpiv=0.05

w0=-1.0
wa=0.0
gamma=0.545

c=3e5

print om0

0.32


In [5]:
#Set up the fiducial cosmology
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy(w0) 
#pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [6]:
#calculate results for these parameters
results = camb.get_results(pars)
#print pars

In [7]:
#Get matter power spectrum at z=0: P(k,z=0)

#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)

#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

In [8]:
#Construct P(k,z=0) interpolating function, in units of Mpc (i.e. no h factors)
Pkz0 = interp1d(kh*hubble, pk[0]/pow(hubble,3))

In [9]:
#get fiducial σ8 (at z=0)
sig8 = results.get_sigma8()[0]
print sig8

0.830287480109


In [10]:
#Redshift bins
zlist = np.arange(0.7,2.1,0.1)
ztest = zlist[0]
Nzbins = len(zlist)

biaslist = [1.083, 1.125, 1.104, 1.126, 1.208, 1.243, 1.282, 1.292, 1.363, 1.497, 1.486, \
            1.491, 1.573, 1.568]

kmin = 0.001*hubble
kmax = 0.2*hubble

dn3 = [2434.28, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, \
       2308.975, 1541.831, 1474.707, 893.716, 497.613]

print zlist
print len(zlist), len(dn3)

[ 0.7  0.8  0.9  1.   1.1  1.2  1.3  1.4  1.5  1.6  1.7  1.8  1.9  2. ]
14 14


In [11]:
#Spatially flat Universe

#Define Omega_DE
def OmDE(zc):
    return (1-om0)*np.exp(-3*wa*zc/(1+zc))*pow(1/(1+zc),-3*(1+w0+wa))

#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(OmDE(zc)+om0*pow(1+zc,3))
def Hz(zc):
    return Ez(zc)*H00

#Define the cosmological distances
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)
def DA(zc):
    return rcom(zc)/(1+zc)

print rcom(ztest)
print DA(ztest)

2596.15131772
1527.14783395


In [12]:
#Define the growth function in LCDM
def fg(zz):
    omz=om0*pow(1+zz,3)/pow(Ez(zz),2)
    return pow(omz,gamma)

print fg(ztest)

0.822099034946


In [13]:
#Get the growth factor 
def Dg_dz(zz):
    return -fg(zz)/(1+zz)

def Dgz(zc):
    ans = scipy.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(ans)

print Dgz(ztest), Dgz(0)

0.693913449649 1.0


In [14]:
def sig8z(zc):
    return Dgz(zc)*sig8

print sig8z(ztest)

0.576147649523


In [15]:
#Construct P(k,μ,z) 
def beta(zc):
    return fg(zc)/bg

def Pkz(kk,mu,zc):    
    return pow(1+beta(zc)*mu**2,2)*pow(Dgz(zc),2)*Pkz0(kk)

In [16]:
#Construct P_gg(k,μ,z)
def Pgg(kk,mu,zc):
    return pow(bg,2)*Pkz(kk,mu,zc)

In [17]:
#Euclid
Area=15000.0 #deg^2
omegatot = Area*pow(pi/180,2)

Dzbin = 0.1

In [18]:
#survey (bin) volume [Mpc^3]
def dVsurdz(zz):    
    return omegatot*c*pow(rcom(zz),2)/(H00*Ez(zz))
    
def Vsur(zc):
    return scipy.integrate.romberg(dVsurdz,zc-Dzbin/2,zc+Dzbin/2)

def Pshot(zc):
    return 1/nbar

#effective volume
def Veff(kk,mu,zc):
    return Vsur(zc)*(Pgg(kk,mu,zc)/(Pgg(kk,mu,zc)+Pshot(zc)))**2

print "%.4g" % Vsur(ztest)

9.184e+09


In [19]:
#Fisher matrix parameters 
params = ["0:fsig8","1:bsig8","2:DA","3:H"]

In [20]:
#Fisher matrix derivatives

def dlnP_dlnfsig8(kk,mu,zc):
    return 2*mu**2*fg(zc)*sig8z(zc)/(bg*sig8z(zc)+mu**2*fg(zc)*sig8z(zc))

def dlnP_dlnbsig8(kk,mu,zc):
    return 2*bg*sig8z(zc)/(bg*sig8z(zc)+mu**2*fg(zc)*sig8z(zc))

def dlnP_dlnDA(kk,mu,zc):
    dk = (kmax-kmin)/400
    return (-2.0+4*mu**2*(1-mu**2)*fg(zc)*sig8z(zc)/(bg*sig8z(zc)+mu**2*fg(zc)*sig8z(zc))
            -kk*(1-mu**2)*(1/Pkz0(kk))*(Pkz0(kk+dk)-Pkz0(kk-dk))/(2*dk))

def dlnP_dlnH(kk,mu,zc):
    dk = (kmax-kmin)/400
    return (1.0+4*mu**2*(1-mu**2)*fg(zc)*sig8z(zc)/(bg*sig8z(zc)+mu**2*fg(zc)*sig8z(zc))
            +kk*mu**2*(1/Pkz0(kk))*(Pkz0(kk+dk)-Pkz0(kk-dk))/(2*dk))



#print dlnP_dlnfsig8(0.1,1.0,ztest)
#print dlnP_dlnbsig8(0.1,0.0,ztest)
#print dlnP_dlnDA(0.1,0.5,ztest)
#print dlnP_dlnH(0.1,0.5,ztest)

In [21]:
def dF(kk,mu):
    return (1./(8*pi*pi))*pow(kk,2)*deriv_i(kk,mu,zc)*deriv_j(kk,mu,zc)*Veff(kk,mu,zc)    

In [22]:
#2D integration function
def integrate2D(dfun, kgrid, mugrid):
    
    muint = [scipy.integrate.simps(dfun.T[i], mugrid) for i in range(kgrid.size)]
    return scipy.integrate.simps(muint, kgrid)

In [23]:
mugrid = np.linspace(-1., 1., 200) 

In [24]:
#%%time
#   Fisher matrix   # 

Npar = 4
#create array of zeros
s = (Npar,Npar)

for zi in range(0,Nzbins):
    zc = zlist[zi]
    bg = biaslist[zi]
    zmin = zc-Dzbin/2
    zmax = zc+Dzbin/2
    
    dVdz = (4*pi/3)*((1+zmax)**3*DA(zmax)**3-(1+zmin)**3*DA(zmin)**3)/(4*pi*pow(180/pi,2))
    nbar = Dzbin*dn3[zi]/dVdz
    
    kgrid = np.linspace(kmin, kmax, 400)
    K, MU = np.meshgrid(kgrid, mugrid)
    Fishermat = np.zeros(s)
    for i in range(0,Npar):  
        def deriv_i(kk,mu,zc):
            if i==0:  return dlnP_dlnfsig8(kk,mu,zc)
            elif i==1:  return dlnP_dlnbsig8(kk,mu,zc)
            elif i==2:  return dlnP_dlnDA(kk,mu,zc)
            elif i==3:  return dlnP_dlnH(kk,mu,zc)
            else: print "index out of range"
        for  j in range(0,Npar):
            if j>=i:
                def deriv_j(kk,mu,zc):
                    if j==0:  return dlnP_dlnfsig8(kk,mu,zc)
                    elif j==1:  return dlnP_dlnbsig8(kk,mu,zc)
                    elif j==2:  return dlnP_dlnDA(kk,mu,zc)
                    elif j==3:  return dlnP_dlnH(kk,mu,zc)
                    else: print "index out of range" 
                Fishermat[i][j] = integrate2D(dF(K,MU),kgrid,mugrid)                
            else: Fishermat[i,j] = Fishermat[j,i]
    print zc, np.sqrt(linalg.inv(Fishermat)[0,0]),\
              np.sqrt(linalg.inv(Fishermat)[1,1]),\
              np.sqrt(linalg.inv(Fishermat)[2,2]),\
              np.sqrt(linalg.inv(Fishermat)[3,3])

0.7 0.0144616223722 0.00682532196831 0.00763354391424 0.00936080932424
0.8 0.012598131886 0.00593050120045 0.00660210151396 0.00824342104711
0.9 0.0116229418844 0.00569646688883 0.00634012026509 0.00781327762384
1.0 0.0113548899271 0.00554226895381 0.00617820853242 0.00757764002332
1.1 0.0117692050285 0.00541088800755 0.00605507219082 0.00748181884554
1.2 0.0119635778246 0.00539073839285 0.00605044466235 0.00745061258768
1.3 0.0122975281257 0.00540574903071 0.00608715132831 0.00747713857338
1.4 0.0128011325216 0.00560204595286 0.00633284185088 0.00768139643354
1.5 0.0140748789524 0.00582891431573 0.00663496591596 0.00802880573628
1.6 0.0166462956232 0.00621843942226 0.00716149903305 0.00869225265742
1.7 0.0198385174628 0.00743399707268 0.00864583984552 0.0101389175832
1.8 0.0207572902933 0.00777603899484 0.00906516510429 0.0105413716129
1.9 0.0280194603494 0.00986095021698 0.0116759025803 0.0132894572864
2.0 0.041535397887 0.0146155464508 0.017531049036 0.0191136545266
